In [1]:
import pandas as pd
from time import strptime
import locale

In [21]:
df_newport=pd.read_csv("https://raw.githubusercontent.com/paulineattal/Disney-Text-Mining/main/fichiers/Scrapping_Newport_Bay_Club_071222.csv",sep=';')
#et les autres ...





In [22]:
#fusionnner tous les DF 
df = df_newport 

# table date 


In [28]:
df_mois_sejour=df['reservation_type'].map(str)
for i in range(df.shape[0]):
    df_mois_sejour[i]=df_mois_sejour[i].split()[3]
print(df_mois_sejour)

df_mois_comm=df['date_review'].map(str)
for i in range(df.shape[0]):
    df_mois_comm[i]=df_mois_comm[i].split()[-2]
print(df_mois_comm)

0       Septembre
1        Novembre
2        Novembre
3        Novembre
4        Novembre
          ...    
2280      Janvier
2281     Décembre
2282      Janvier
2283      Janvier
2284     Décembre
Name: reservation_type, Length: 2285, dtype: object
0            des
1       novembre
2       novembre
3       novembre
4       novembre
          ...   
2280     janvier
2281     janvier
2282     janvier
2283     janvier
2284    décembre
Name: date_review, Length: 2285, dtype: object


In [29]:
df_annee_sejour=df['reservation_type'].map(str)
for i in range(df.shape[0]):
    df_annee_sejour[i]=df_annee_sejour[i].split()[4]
print(df_annee_sejour)

df_annee_comm=df['date_review'].map(str)
for i in range(df.shape[0]):
    df_annee_comm[i]=df_annee_comm[i].split()[-1]
print(df_annee_comm)

0       2022
1       2022
2       2022
3       2022
4       2022
        ... 
2280    2020
2281    2019
2282    2020
2283    2020
2284    2019
Name: reservation_type, Length: 2285, dtype: object
0       voyageurs
1            2022
2            2022
3            2022
4            2022
          ...    
2280         2020
2281         2020
2282         2020
2283         2020
2284         2019
Name: date_review, Length: 2285, dtype: object


In [25]:
locale.setlocale(locale.LC_TIME,'')
list_mois_num = [strptime(mois_sejour,'%B').tm_mon for mois_sejour in df_mois_sejour]

d = {'mois_str': df_mois_sejour, 'mois_num': list_mois_num, 'année' : df_annee_sejour}
df_date = pd.DataFrame(data=d)
df_date

#possibilité de rajouter num semaine, une vraie date au format, ou autres info ?? 


,mois_str,mois_num,année
0,Septembre,9,2022
1,Novembre,11,2022
2,Novembre,11,2022
3,Novembre,11,2022
4,Novembre,11,2022
...,...,...,...
2280,Janvier,1,2020
2281,Décembre,12,2019
2282,Janvier,1,2020
2283,Janvier,1,2020


In [26]:
df

,Names,Country,room_type,reservation_type,Duration,traveler_infos,date_review,review_title,grade_review,positive_review,negative_review,usefulness_review,hotel
0,Ganna,Ukraine,Chambre Supérieure - Côté Lac,2 nuits · Septembre 2022,Septembre 2022,Famille,Le choix des voyageurs,Good choice for visiting Disneyland,10,NaN,NaN,Utile Pas utile,Disney's Newport Bay Club
1,Diego,France,Chambre Supérieure,1 nuit · Novembre 2022,Novembre 2022,Famille,Commentaire envoyé le 30 novembre 2022,Agréable,"6,0",la proximité du parc Disneyland Paris,"la qualité de la chambre n’était pas top, vieu...",Utile Pas utile,Disney's Newport Bay Club
2,Fabienne,Belgique,Chambre Supérieure,1 nuit · Novembre 2022,Novembre 2022,Famille,Commentaire envoyé le 18 novembre 2022,déçue qualité prix,"6,0",l emplacement,la propreté,Utile Pas utile,Disney's Newport Bay Club
3,Pierre,France,Chambre Supérieure,1 nuit · Novembre 2022,Novembre 2022,Famille,Commentaire envoyé le 14 novembre 2022,très moyen par rapport au prix,"6,0",très bon personnel dédié mais la sécurité fait...,hôtel très vieillissant et pas de shampoing ge...,Utile Pas utile,Disney's Newport Bay Club
4,Christine,France,Chambre Supérieure,1 nuit · Novembre 2022,Novembre 2022,Famille,Commentaire envoyé le 10 novembre 2022,Très bien,"8,0",La décoration de la chambre est adorable.\nOn ...,La porte ne marchait pas bien.\nQuand on met l...,Utile Pas utile,Disney's Newport Bay Club
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2280,Anonyme,Brésil,NaN,2 nuits · Janvier 2020,Janvier 2020,Famille,Commentaire envoyé le 14 janvier 2020,Exceptionnel,10,NaN,NaN,NaN,Disney's Newport Bay Club
2281,Anonyme,Royaume-Uni,NaN,2 nuits · Décembre 2019,Décembre 2019,Couple,Commentaire envoyé le 14 janvier 2020,Fabuleux,"9,2",NaN,NaN,NaN,Disney's Newport Bay Club
2282,Anonyme,Portugal,NaN,2 nuits · Janvier 2020,Janvier 2020,Couple,Commentaire envoyé le 13 janvier 2020,Fabuleux,"9,0",NaN,NaN,NaN,Disney's Newport Bay Club
2283,Anonyme,Japon,NaN,3 nuits · Janvier 2020,Janvier 2020,Famille,Commentaire envoyé le 8 janvier 2020,Décevant,"4,2",NaN,NaN,NaN,Disney's Newport Bay Club


In [ ]:
# Liste des colonnes à transtyper en numeric
var_tofloat = ['grade_review']
for i in var_tofloat:
    df[i] = df[i].str.replace(",",".")
    df[i] = pd.to_numeric(df[i], downcast="float")

In [ ]:
# Liste des colonnes à transtyper en numeric
var_toint = ['nuitee']
for i in var_toint:
    df[i] = df[i].astype('int')

In [ ]:
d = {'mois_str': df_mois_sejour, 'mois_num': list_mois_num, 'année' : df_annee_sejour}
df_client = pd.DataFrame(data=d)
df_client

In [ ]:
df.to_csv("https://raw.githubusercontent.com/paulineattal/Disney-Text-Mining/main/fichiers/df_clean.csv",sep=';', header=False)